In [1]:
from pandas import read_csv,DataFrame,read_excel, merge,concat, Series
import os
import re
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import gaussian_kde
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [2]:
datacsv          = "peru_ap17/"
data_peru        = "peru_data/"
list_csv         = os.listdir(datacsv)

In [3]:
baseline = read_csv(datacsv+"OD_baseline_Peru.csv")
baseline.index = baseline.Name
baseline["O"] = baseline.Name.apply(lambda s: re.search("(.*) - (.*)",s).group(1))
baseline["D"] = baseline.Name.apply(lambda s: re.search("(.*) - (.*)",s).group(1))

# Initial analysis: "naive" calculation of costs

In [4]:
if os.path.exists("costs_peru_ap22.csv"):
    costs_all = read_csv("costs_peru_ap22.csv")
else:
    costs_all = DataFrame(columns=["scenarioID","diff ruc from baseline (%)","diff ruc from baseline (usd)",\
                               "missingroutes",\
                               "number of affected routes","av cost with traffic (usd)",\
                               "max cost with traffic (%)","max cost with traffic (usd)",\
                               "av km difference","max km difference","tot km difference"])
    for file in list_csv:
        if ".csv" not in file:
            continue
        if file=="OD_baseline_Peru.csv":
            continue
        if "Partial" in file:
            continue
        scenar = read_csv(datacsv+file)
        scenar.index = scenar.Name
        n      = int(re.search('OD_scenario(.*).csv', file).group(1))
        ruc    = scenar.Total_Ta_r.sum()
        voc    = scenar.Total_Ta_v.sum()
        missingroutes = len(baseline)-len(scenar)
        baseline_nomissingroutes = baseline.ix[[i in scenar.index for i in baseline.index],:]
        diff_from_baseline = 100*(ruc/baseline_nomissingroutes.Total_Ta_r.sum()-1)
        diff_from_baseline_abs = ruc-baseline_nomissingroutes.Total_Ta_r.sum()
        diff_km_from_baseline  = scenar.Total_KM.sum()-baseline_nomissingroutes.Total_KM.sum()
        diff_km = scenar.Total_KM - baseline_nomissingroutes.Total_KM
        affected_routes = np.round(scenar.Total_Ta_r/baseline_nomissingroutes.Total_Ta_r,3)!=1
        af_cost = 100*(scenar.ix[affected_routes,"Total_Ta_r"]*baseline_nomissingroutes.ix[affected_routes,"avg_TPDA"]\
        /(baseline_nomissingroutes.ix[affected_routes,"Total_Ta_r"]*baseline_nomissingroutes.ix[affected_routes,"avg_TPDA"])-1)
        af_cost_abs = scenar.ix[affected_routes,"Total_Ta_r"]*baseline_nomissingroutes.ix[affected_routes,"avg_TPDA"]\
        -(baseline_nomissingroutes.ix[affected_routes,"Total_Ta_r"]*baseline_nomissingroutes.ix[affected_routes,"avg_TPDA"])

        costs_all.loc[len(costs_all),:] = [n,diff_from_baseline,diff_from_baseline_abs,\
                                   missingroutes,sum(affected_routes),\
                                   np.mean(af_cost_abs),np.max(af_cost),np.max(af_cost_abs),\
                                  np.mean(diff_km),np.max(diff_km),diff_km_from_baseline]

    costs_all.to_csv("costs_peru_ap22.csv",index=False)

# Second analysis: calculation of costs with weights for nodes based on the gravity model

### calculates weights for each node

In [5]:
nodes1 = read_excel(data_peru+"NODE_CID.xlsx","NODES_CID")
nodes2 = read_excel(data_peru+"NODES_10km_TPDA.xlsx","NODES_10km_TPDA")

nodes = merge(nodes1, nodes2, on='CID', how='outer')

#### the function below calculates the weight of a OD pair based on the gravity model

In [6]:
baseline["Ot"] = baseline.O.replace(nodes.set_index('CID').Sum_TPDA)
baseline["Dt"] = baseline.D.replace(nodes.set_index('CID').Sum_TPDA)
baseline["Opop"] = baseline.O.replace(nodes.set_index('CID')["Population Headcount"])
baseline["Dpop"] = baseline.D.replace(nodes.set_index('CID')["Population Headcount"])

#### here we calculate losses for each scenario

In [7]:
baseline.ix[baseline.Total_KM!=0,'weights']=baseline.ix[baseline.Total_KM!=0,'Ot']\
                                            *baseline.ix[baseline.Total_KM!=0,'Dt']\
                                            /baseline.ix[baseline.Total_KM!=0,'Total_KM']**2
baseline.weights.fillna(0,inplace=True)

In [8]:
info_links = read_excel(data_peru+"Traffic_Link_Final.xlsx","Traffic_Link_Final")
#info_links = read_excel(datacsv+"allinfo.xlsx","Traffic_Link_AllInfo")

In [9]:
if os.path.exists("costs_peru_may12.csv"):
    costs_all = read_csv("costs_peru_may12.csv")
else:
    costs_all = DataFrame(columns=["scenarioID","partial_or_full","part_ruc_increase","ruc_increase","missingroutes",\
                               "num_aff_routes","cost_with_traffic","km_diff"])
    for file in list_csv:
        if ".csv" not in file:
            continue
        if file=="OD_baseline_Peru.csv":
            continue
        if ("cluster" in file):
            continue
        if ("Partial_V2" in file):
            partial_or_full="partial"
            n      = int(re.search('Scenario_OD(.*)_Partial_V2.csv', file).group(1))
            part_ruc_increase = 0.05
        elif ("Partial_v1" in file):
            partial_or_full="partial"
            n      = int(re.search('Scenario_OD(.*)_Partial_v1.csv', file).group(1))
            part_ruc_increase = 0.5
        else:
            partial_or_full='full'
            n      = int(re.search('OD_scenario(.*).csv', file).group(1))
            part_ruc_increase = 0.
        scenar = read_csv(datacsv+file)
        scenar.index = scenar.Name
        
        missingroutes = len(baseline)-len(scenar)
        # we do not take into account the routes that don't have a second best solution
        baseline_nm = baseline.ix[[i in scenar.index for i in baseline.index],:]
        # we select only routes that get affected by the disruption
        affected_routes = (np.round(scenar.Total_Ta_r/baseline_nm.Total_Ta_r,3)!=1)&(baseline_nm.Total_Ta_r>0)
        subscenar = scenar.ix[affected_routes,:]
        subscenar["weights"] = baseline_nm.weights
        
        traffic = info_links.ix[info_links.ScenarioID,"TPDA"].values[0]
            
        diff_ruc_baseline   = (subscenar.Total_Ta_r-\
                               baseline_nm.ix[affected_routes,"Total_Ta_r"])
        diff_km_from_baseline = (subscenar.Total_KM-baseline_nm.ix[affected_routes,"Total_KM"])
        diff_tot_baseline   = (traffic*diff_ruc_baseline)
        
        ruc_increase      = np.sum(diff_ruc_baseline*subscenar.weights)/subscenar.weights.sum()
        km_diff = np.sum(diff_km_from_baseline*subscenar.weights)/subscenar.weights.sum()
        cost_with_traffic  = np.sum(diff_tot_baseline*subscenar.weights)/subscenar.weights.sum()

        costs_all.loc[len(costs_all),:] = [n,partial_or_full,part_ruc_increase,\
                                           ruc_increase,missingroutes,sum(affected_routes),\
                                           cost_with_traffic,km_diff]

    costs_all.to_csv("costs_peru_may12.csv",index=False)

here I merge the info that CJ sent with the results of the analysis (calculation of disruption costs)

In [10]:
allinfo = merge(info_links, costs_all.rename(columns={"scenarioID":"ScenarioID"}), on='ScenarioID', how='inner')
allinfo.index=allinfo.ScenarioID

# Clusters of links

In [11]:
cluster1 = np.array([i in [195, 567, 568, 569, 570, 572, 719, 720] for i in allinfo.ScenarioID])
cluster2 = np.array([i in [135,136,138,139,140,545,654] for i in allinfo.ScenarioID])
cluster3 = np.array([i in [124, 126, 131, 622, 623, 624, 642, 643, 645]+list(range(594,621))\
                     for i in allinfo.ScenarioID])
cluster4 = np.array([i in [271,272] for i in allinfo.ScenarioID])

In [12]:
costs_clusters = DataFrame(columns=["ScenarioID","partial_or_full",'part_ruc_increase',"ruc_increase","missingroutes",\
                           "num_aff_routes","cost_with_traffic","km_diff"])
for file in list_csv:
    if ".csv" not in file:
        continue
    if "cluster" not in file:
        continue
    if ("partial" in file):
        partial_or_full="partial"
        n      = int(re.search('OD_scenario_cluster(.*)_partial.csv', file).group(1))
        part_ruc_increase = 0.05
    else:
        partial_or_full="full"
        n      = int(re.search('OD_scenario_cluster(.*).csv', file).group(1))
        part_ruc_increase = 0.
    scenar = read_csv(datacsv+file)
    scenar.index = scenar.Name
    

    missingroutes = len(baseline)-len(scenar)
    # we do not take into account the routes that don't have a second best solution
    baseline_nm = baseline.ix[[i in scenar.index for i in baseline.index],:]
    # we select only routes that get affected by the disruption
    affected_routes = (np.round(scenar.Total_Ta_r/baseline_nm.Total_Ta_r,3)!=1)&(baseline_nm.Total_Ta_r!=0)
    subscenar = scenar.ix[affected_routes,:]
    subscenar["weights"] = baseline_nm.ix[affected_routes,'weights']

    traffic = info_links.ix[info_links.ScenarioID,"TPDA"].values[0]

    diff_ruc_baseline   = (subscenar.Total_Ta_r-\
                           baseline_nm.ix[affected_routes,"Total_Ta_r"]).dropna()
    diff_km_from_baseline = (subscenar.Total_KM-\
                             baseline_nm.ix[affected_routes,"Total_KM"]).dropna()
    diff_tot_baseline   = (traffic*diff_ruc_baseline).dropna()
    
    if sum(affected_routes)>0:
        ruc_increase      = np.sum(diff_ruc_baseline*subscenar.weights)/subscenar.weights.sum()
        km_diff = np.sum(diff_km_from_baseline*subscenar.weights)/subscenar.weights.sum()
        cost_with_traffic  = np.sum(diff_tot_baseline*subscenar.weights)/subscenar.weights.sum()
    else:
        ruc_increase      = 0
        km_diff = 0
        cost_with_traffic  = 0

    costs_clusters.loc[len(costs_clusters),:] = ["cluster{}".format(n),partial_or_full,part_ruc_increase,\
                                                 ruc_increase,missingroutes,sum(affected_routes),\
                                       cost_with_traffic,km_diff]

In [13]:
allinfo = allinfo.append(costs_clusters,ignore_index=True)

In [14]:
for clu in ['cluster2', 'cluster3', 'cluster4']:

    for col in ['ADMIN_NAME', 'CITY_NAME', 'CLASS', 'CNTRY_NAME', 'CODIGO', 'COND1', 'CORR_ID','Identifier',\
               'LANES', 'NAME_0', 'OD', 'OPTIMAL', 'STATUS', 'STATUS.1', 'SURFACE1','TERRAIN', 'TID',]:
        allinfo.ix[allinfo.ScenarioID==clu,col]=allinfo.ix[eval(clu),col].iloc[0]

    for climat in ['EU_historical','GFDL_8.5','HadGEM2_8.5','IPSL_8.5']:
        for RP in [5,10,25,50,100,250,500,1000]:
            col = "{}_RP{} (cm)".format(climat,RP)
            allinfo.ix[allinfo.ScenarioID==clu,col]=allinfo.ix[eval(clu),col].max()

    for col in ["KM"]:
        allinfo.ix[allinfo.ScenarioID==clu,col]=allinfo.ix[eval(clu),col].sum()

    for col in ['Elev_Max (m)','TPDA']:
        allinfo.ix[allinfo.ScenarioID==clu,col]=allinfo.ix[eval(clu),col].max()

    for col in ['Elev_Min (m)']:
        allinfo.ix[allinfo.ScenarioID==clu,col]=allinfo.ix[eval(clu),col].min()

In [15]:
cluster1 = np.array([i in [195, 567, 568, 569, 570, 572, 719, 720] for i in allinfo.ScenarioID])
cluster2 = np.array([i in [135,136,138,139,140,545,654] for i in allinfo.ScenarioID])
cluster3 = np.array([i in [124, 126, 131, 622, 623, 624, 642, 643, 645]+list(range(594,621))\
                     for i in allinfo.ScenarioID])
cluster4 = np.array([i in [271,272] for i in allinfo.ScenarioID])

In [18]:
allinfo.to_csv("allinfo_peru.csv",index=False,encoding='utf-8')

In [25]:
allinfo.ix[allinfo.ScenarioID=="cluster2",["cost_with_traffic","partial_or_full","num_aff_routes"]]

,cost_with_traffic,partial_or_full,num_aff_routes
968,237919.1,full,412
969,3723.436,partial,64


In [24]:
allinfo.ix[cluster2,["cost_with_traffic","partial_or_full","num_aff_routes"]]

,cost_with_traffic,partial_or_full,num_aff_routes
137,445102.2,full,124
138,445102.2,full,124
140,237919.1,full,412
141,445102.2,full,124
142,445102.2,full,124
551,237919.1,full,412
660,445102.2,full,124
